In [1]:
import torch
from gpytorch.kernels import RBFKernel
from gpytorch.utils.contour_integral_quad import sqrt_matmul
from gpytorch.lazy import NonLazyTensor

/home/jrg365/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [24]:
x = torch.randn(100, 4)  # data
a = torch.randn(100, 1)  # rhs

In [25]:
kern = RBFKernel(ard_num_dims=4)
K = kern(x).add_jitter(1e-3).evaluate()

In [26]:
retr = K.symeig(eigenvectors=True)
U = retr.eigenvectors
S = retr.eigenvalues
Khalf = U @ ((S.sqrt().unsqueeze(-1)) * U.t())
loss1 = (a.t() @ Khalf @ a)
loss1.backward()

In [27]:
kern.raw_lengthscale.grad

tensor([[ -9.7001,  -6.3232,  -2.0402, -10.9935]])

In [28]:
kern.zero_grad()
K = kern(x).add_jitter(1e-3).evaluate()
Klt = NonLazyTensor(K)
ciq_khalf_a = sqrt_matmul(Klt, a)
loss2 = a.t() @ ciq_khalf_a
loss2.backward()

In [29]:
kern.raw_lengthscale.grad

tensor([[ -9.7003,  -6.3234,  -2.0405, -10.9937]])

In [10]:
torch.norm(ciq_khalf_a - (Khalf @ a).squeeze()) / ciq_khalf_a.norm()

tensor(1.2763e-06, grad_fn=<DivBackward0>)

In [11]:
ciq_khalf_a

tensor([-1.0233,  0.4011,  0.3243, -0.5014, -0.4869, -0.4091, -0.8845,  1.0060,
         1.2145,  1.9591,  0.7357,  1.0515,  0.1106,  0.1897, -0.9039,  0.9867,
         0.0098, -0.2459, -1.8282, -0.4605, -0.3332, -0.6090, -0.2521,  0.4092,
         0.3234, -0.3297, -0.3069, -0.3155, -0.3201, -0.9143,  0.7945, -1.2578,
        -1.0009,  0.4052, -1.1673,  0.0495,  0.2226,  1.0111,  0.3790,  0.9021,
        -0.7298, -0.1216, -0.7699, -0.9955, -0.8892,  0.2117, -0.1805, -0.3667,
        -0.5813, -1.0053, -1.2286,  1.1297,  0.4926,  0.0952,  0.6485,  0.3180,
         0.6567, -0.7234, -0.3450, -0.2421,  1.5285, -1.3605,  0.0782, -0.2534,
        -0.3581,  0.8362,  1.1073,  0.2851,  0.8625,  0.2379,  0.5609, -0.8899,
         0.0985, -1.5315,  0.0685, -1.1257,  0.3577, -1.0338, -0.3735, -0.0255,
         0.5237,  0.3266, -0.1217,  0.3944,  0.4579, -0.2042, -1.2395, -0.4996,
        -1.2028,  1.0056,  1.1855, -0.6769, -1.7770,  0.5499,  0.2285, -0.9464,
         1.1737,  0.4801, -0.1574,  0.99

In [12]:
K = kern(x).add_jitter(1e-3).evaluate()

solve1 = torch.cholesky_solve(a, torch.cholesky(K))
solve2 = torch.solve(a, K).solution

lss1 = a.t() @ solve1
lss2 = a.t() @ solve2